In [1]:
from ogb.nodeproppred import NodePropPredDataset
from sklearn.model_selection import ParameterGrid
%reload_ext autoreload
%autoreload 2
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import convo as c 
import convL 
import modularized as m 
import batching as b 
import poolingL
import gin
import math 
import funcy as fy
import gridSearch as g 
import json 
import os

/home/olga/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-15 10:40:49.242943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 10:40:51.635199: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-15 10:40:51.635575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [7]:
import sys
sys.path.append("/content/drive/Othercomputers/Mein PC/Completed")

In [9]:
dataset3 = NodePropPredDataset(name='ogbn-products')

Downloaded 1.38 GB: 100%|██████████| 1414/1414 [23:08<00:00,  1.02it/s]


Extracting dataset\products.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:05<00:00,  5.51s/it]


Saving...


In [4]:
# create model with layers
dataset1 = b.GraphPropPredDataset(name='ogbg-molhiv')


In [2]:
param_grid = {'num_layers': [1, 2, 3, 4, 5, 6],
              'learning_rate': [0.0001, 0.001, 0.01],
              'optimizer': ['Adam'],
              'regularization': ['DropOut', 'NodeSampling', 'DropEdge', 'GDC'],
              'probability': [0.3, 0.5, 0.7],
              'activation': ['relu', 'sigmoid', 'tanh'],
              'units': [32, 64],
              'convo_type': ['gcn', 'gin']}


sample = list(ParameterGrid(param_grid))[0:5]
big_sample = list(ParameterGrid(param_grid))[0:20]


In [32]:
concrete_inputs = b.make_tf_datasets(dataset1)
training = concrete_inputs['train']

In [33]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=50)
history = list_models[0].fit(training.cache(), epochs  =100, callbacks = callback)


Epoch 1/100
    136/Unknown - 5s 16ms/step - loss: 1.5176WARNING:tensorflow:Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss
138/138 [==============================] - 5s 17ms/step - loss: 1.5107
Epoch 2/100
138/138 [==============================] - 2s 18ms/step - loss: 0.3528
Epoch 3/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1879
Epoch 4/100
138/138 [==============================] - 2s 16ms/step - loss: 0.1544
Epoch 5/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1442
Epoch 6/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1416
Epoch 7/100
138/138 [==============================] - 2s 16ms/step - loss: 0.1412
Epoch 8/100
138/138 [==============================] - 2s 11ms/step - loss: 0.1398
Epoch 9/100
138/138 [==============================] - 2s 14ms/step - loss: 0.1393
Epoch 10/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1400
Epoch

In [32]:
g.create_grid_models(sample, dataset1, 'ogbg-molhiv')

IndexError: Only integer is valid index (got str).

In [11]:
from aim import Run
from aim import Run 
run = Run()

# aim_test.py

run = Run()

# set training hyperparameters
run['hparams'] = {
    'learning_rate': 0.001,
    'batch_size': 32,
}

# log metric
for i in range(10):
    run.track(i, name='numbers')


In [12]:
type(run)

aim.sdk.run.Run

In [23]:
my_batch = b.make_tf_datasets(dataset=dataset1)
my_batch['train']

<FlatMapDataset element_spec=({'X': TensorSpec(shape=(None, 9), dtype=tf.float32, name=None), 'ref_A': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), 'ref_B': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), 'num_nodes': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>